# Lag Selection for Dynamic Factor Model

This notebook implements a systematic approach to select the optimal number of lags and factors for Dynamic Factor Models (DFM). The lag selection process is performed on rolling splits of the data for three regions: Switzerland (CH), the Euro Area (EU), and the United States (US).

In [ ]:
# Add the root directory of the repo to sys.path
import sys
from pathlib import Path

ROOT_DIR = Path("../").resolve()
sys.path.append(str(ROOT_DIR))

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm
from src.dfm_utils.helpers import preprocess_multivar, dynamic_factors, bai_ng_criteria, onatski_criterion

In [ ]:
# Load datasets for each region
DATA_DIR = ROOT_DIR / "data" / "processed"
datasets = {
    "ch": pd.read_csv(DATA_DIR / "ch_data_final.csv"),
    "eu": pd.read_csv(DATA_DIR / "eu_data_final.csv"),
    "us": pd.read_csv(DATA_DIR / "us_data_final.csv"),
}

datasets_win = {
    "ch": pd.read_csv(DATA_DIR / "ch_data_transformed_win.csv"),
    "eu": pd.read_csv(DATA_DIR / "eu_data_transformed_win.csv"),
    "us": pd.read_csv(DATA_DIR / "us_data_transformed_win.csv"),
}

# Define variable sets for each region (compact)
variable_sets = {
    "ch": [
        "cpi_total_yoy", "cpi_goods_cat_goods_ind", "cpi_goods_cat_services_ind", "cpi_housing_energy_ind",
        "cpi_food_nonalcoholic_beverages_ind", "cpi_transport_ind", "cpi_health_ind", "cpi_clothing_footwear_ind",
        "cpi_alcoholic_beverages_tobacco_ind", "cpi_household_furniture_furnishings_routine_maintenance_ind",
        "cpi_restaurants_hotels_ind", "cpi_recreation_culture_ind", "cpi_communications_ind", "cpi_education_ind",
        "mon_stat_mon_agg_m0_total_chf", "ppi_total_base_month_december_2020_ind", "ipi_total_base_month_december_2020_ind",
        "oilpricex"
    ],
    "eu": [
        "hcpi_yoy", "irt3m_eacc", "irt6m_eacc", "ltirt_eacc", "ppicag_ea", "ppicog_ea", "ppindcog_ea", "ppidcog_ea",
        "ppiing_ea", "ppinrg_ea", "hicpnef_ea", "hicpg_ea", "hicpin_ea", "hicpsv_ea", "hicpng_ea", "curr_eacc",
        "m2_eacc", "m1_eacc", "oilpricex"
    ],
    "us": [
        "cpi_all_yoy", "m1sl", "m2sl", "m2real", "busloans", "fedfunds", "tb3ms", "tb6ms", "gs1", "gs5", "gs10",
        "ppicmm", "oilpricex", "cpiappsl", "cpitrnsl", "cpimedsl", "cusr0000sac", "cusr0000sad", "cusr0000sas", "pcepi"
    ],
}

# Define transformed variable sets for each region
variable_sets_t = {
    region: [f"{var}_t" for var in variables]
    for region, variables in variable_sets.items()
}

# Load multivariate data
multivar_data = {}
for region, columns in variable_sets.items():
    multivar_data[region] = preprocess_multivar(datasets[region], columns)
    print(
        f"{region.upper()}: {multivar_data[region].shape} - columns: {list(multivar_data[region].columns)}"
    )
    print(
        f"{region.upper()} time series range: {multivar_data[region].index[0].strftime('%Y-%m-%d')} to {multivar_data[region].index[-1].strftime('%Y-%m-%d')}"
    )

# And winsorized data
multivar_data_win = {}
for region, columns in variable_sets_t.items():
    multivar_data_win[region] = preprocess_multivar(datasets_win[region], columns)
    print(
        f"{region.upper()} winsorized: {multivar_data_win[region].shape} - columns: {list(multivar_data_win[region].columns)}"
    )
    print(
        f"{region.upper()} winsorized time series range: {multivar_data_win[region].index[0].strftime('%Y-%m-%d')} to {multivar_data_win[region].index[-1].strftime('%Y-%m-%d')}"
    )

In [ ]:
# --- Scree Plots and Data Storage ---
regions = ['ch', 'eu', 'us']
split_percentages = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
variance_thresholds = {
    '50%': 0.50,
    '75%': 0.75,
    '80%': 0.80,
    '90%': 0.90
}
threshold_colors = {'50%': 'green', '75%': 'orange', '80%': 'blue', '90%': 'red'}

# Initialize a list to store the results for the DataFrame
results_rows = []

# --- Main Loop to Generate Plots and Collect Data ---
for region in regions:
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle(f'Scree Plots for {region.upper()} Region by Training Split', fontsize=18)
    axes = axes.flatten()

    data_matrix = multivar_data_win[region].copy()
    data_matrix = data_matrix.select_dtypes(include=[np.number]).dropna(axis=1, how='all')
    T_total, N_total = data_matrix.shape

    for i, sp in enumerate(split_percentages):
        ax = axes[i]
        split_size = int(T_total * sp)
        Y_train = data_matrix.iloc[:split_size, :].dropna(axis=0)
        
        if Y_train.shape[0] < 2 or Y_train.shape[1] < 2:
            ax.set_title(f"Split {sp*100:.0f}%: Not enough data")
            continue

        scaler = StandardScaler()
        Y_scaled = scaler.fit_transform(Y_train)
        pca = PCA(n_components=Y_scaled.shape[1])
        pca.fit(Y_scaled)
        cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

        ax.plot(range(1, len(cumulative_variance) + 1), pca.explained_variance_ratio_, 'o-', label='Individual Variance')
        ax.set_title(f'Split: {sp*100:.0f}% (T={Y_train.shape[0]}, N={Y_train.shape[1]})')
        ax.set_xlabel('Number of Factors')
        ax.set_ylabel('Proportion of Variance Explained')
        ax.grid(True)
        ax.set_xticks(range(1, len(cumulative_variance) + 1, 2))
        ax.set_xticklabels(range(1, len(cumulative_variance) + 1, 2))

        for label, threshold in variance_thresholds.items():
            num_factors_needed = np.argmax(cumulative_variance >= threshold) + 1
            results_rows.append({
                "region": region, "split": sp, "threshold": label, "num_factors_needed": num_factors_needed
            })
            if num_factors_needed > 0:
                ax.axvline(x=num_factors_needed, color=threshold_colors[label], linestyle='--', 
                           label=f'{label} Var. ({num_factors_needed} factors)')
        ax.legend()

    for j in range(len(split_percentages), len(axes)):
        fig.delaxes(axes[j])
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# --- Display Detailed Results per Region ---
print("\n--- Number of Factors Needed per Split ---")

# Convert results to a DataFrame
nf_analysis_df = pd.DataFrame(results_rows)

# Loop through each region and create a pivot table for it
for region in regions:
    print(f"\n--- {region.upper()} ---")
    
    # Filter the DataFrame for the current region
    region_df = nf_analysis_df[nf_analysis_df['region'] == region]
    
    # Create the pivot table with splits as rows and thresholds as columns
    split_pivot = region_df.pivot_table(
        index='split', 
        columns='threshold', 
        values='num_factors_needed'
    )
    
    # Reorder columns for logical presentation
    split_pivot = split_pivot[['50%', '75%', '80%', '90%']]
    
    print(split_pivot)

In [ ]:
# --- Create the Multi-Index Lookup Series for NF ---
try:
    nf_lookup = nf_analysis_df.set_index(['region', 'split', 'threshold'])['num_factors_needed']
    variance_thresholds = nf_analysis_df['threshold'].unique()
    print("Successfully created NF lookup table from your analysis.")
    print(f"Testing for thresholds: {list(variance_thresholds)}")
except NameError:
    print("ERROR: The 'nf_analysis_df' DataFrame was not found.")
    print("Please run the previous cell to generate the scree plot data first.")
    exit()

# --- Run the Full Sensitivity Analysis for Dynamic Factors ---
regions = ['ch', 'eu', 'us']
split_percentages = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
p_lags = [1, 2, 3]
results_rows = []

# Set up the progress bar with the total number of iterations
total_iterations = len(regions) * len(split_percentages) * len(variance_thresholds) * len(p_lags)
with tqdm(total=total_iterations, desc="Full Analysis") as pbar:
    for region in regions:
        data_matrix = multivar_data_win[region].copy()
        data_matrix = data_matrix.select_dtypes(include=[np.number]).dropna(axis=1, how='all')
        T_total = data_matrix.shape[0]

        for sp in split_percentages:
            split_size = int(T_total * sp)
            Y_train = data_matrix.iloc[:split_size, :].dropna(axis=0)
            T, N = Y_train.shape
            
            if N < 2:
                pbar.update(len(variance_thresholds) * len(p_lags))
                continue

            # Loop over all the variance thresholds
            for threshold in variance_thresholds:
                
                # Look up the correct NF for this specific combination
                try:
                    NF = nf_lookup.loc[(region, sp, threshold)]
                except KeyError:
                    pbar.update(len(p_lags))
                    continue # Skip if this combo doesn't exist

                # Create the scaler
                scaler = StandardScaler(with_mean=True, with_std=False).fit(Y_train.values)
                
                # Loop over VAR lags p
                for p in p_lags:
                    if T <= max(NF, p) + 5: # Ensure T is large enough for NF factors and p lags
                        pbar.update(1)
                        continue
                    
                    # Determine dynamic factors with the specific NF
                    q1, q2 = dynamic_factors(Y_train.values, NF=NF, p=p, scaler=scaler)
                    q = min(q1, q2)

                    results_rows.append({
                        "region": region,
                        "split": sp,
                        "threshold": threshold, # Store which threshold was used
                        "NF_used": NF,
                        "p": p,
                        "q_chosen": q,
                    })
                    pbar.update(1)

# --- Analyze and Display the Comprehensive Results ---
dynamic_ng_df_full_analysis = pd.DataFrame(results_rows)

print("\n--- Sensitivity Analysis Results ---")

for r in dynamic_ng_df_full_analysis["region"].unique():
    print(f"\n--- Region: {r.upper()} ---")
    
    region_df = dynamic_ng_df_full_analysis[dynamic_ng_df_full_analysis.region == r]
    
    # Create a pivot table showing q_chosen for each scenario
    final_pivot = region_df.pivot_table(
        index=['split', 'threshold', 'NF_used'], 
        columns='p', 
        values='q_chosen'
    )
    print(final_pivot)

# Save the results to be used in the model
TARGET_THRESHOLD = '75%'
TARGET_P_LAG = 1

# Filter the full analysis DataFrame
factor_selection = dynamic_ng_df_full_analysis[
    (dynamic_ng_df_full_analysis['threshold'] == TARGET_THRESHOLD) &
    (dynamic_ng_df_full_analysis['p'] == TARGET_P_LAG)
].copy()

# Select and rename the final columns
final_columns = {
    'region': 'region',
    'split': 'split',
    'NF_used': 'static_factors',
    'q_chosen': 'dynamic_factors'
}
factor_selection = factor_selection[final_columns.keys()].rename(columns=final_columns)

print(f"\n Optimal factors for threshold {TARGET_THRESHOLD} and p_lag {TARGET_P_LAG} using Scree Plots:")
print(factor_selection)

In [ ]:
# --- Bai & Ng (2002) Static Factor Analysis ---
regions = ['ch', 'eu', 'us']
split_percentages = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
results_rows_static = []

for region in regions:
    data_matrix = multivar_data_win[region].select_dtypes(include=[np.number]).dropna(axis=1, how='all')
    T_total, N_total = data_matrix.shape

    for sp in split_percentages:
        split_size = int(T_total * sp)
        Y_train = data_matrix.iloc[:split_size, :].dropna(axis=0)

        if Y_train.shape[0] < 2 or Y_train.shape[1] < 2:
            continue

        scaler = StandardScaler()
        Y_scaled = scaler.fit_transform(Y_train)
        
        # Calculate optimal number of factors using Bai & Ng criteria
        _, optimal_factors = bai_ng_criteria(Y_scaled, max_factors=Y_scaled.shape[1] - 1)
        
        # Store results
        result = {
            "region": region,
            "split": sp,
            "T": Y_train.shape[0],
            "N": Y_train.shape[1],
        }
        result.update(optimal_factors)
        results_rows_static.append(result)

# Convert results to a DataFrame for analysis and lookup
bn_analysis_df = pd.DataFrame(results_rows_static).set_index(['region', 'split'])
print("\n--- Optimal Static Factors (k) from Bai & Ng (2002) ---")
print(bn_analysis_df)

In [ ]:
# --- Onatski (2010) Static Factor Analysis ---
regions = ['ch', 'eu', 'us']
split_percentages = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
onatski_results_rows = []

for region in regions:
    data_matrix = multivar_data_win[region].select_dtypes(include=[np.number]).dropna(axis=1, how='all')
    T_total, N_total = data_matrix.shape

    for sp in split_percentages:
        split_size = int(T_total * sp)
        Y_train = data_matrix.iloc[:split_size, :].dropna(axis=0)

        if Y_train.shape[0] < 2 or Y_train.shape[1] < 2:
            continue

        scaler = StandardScaler()
        Y_scaled = scaler.fit_transform(Y_train)
        
        num_factors_onatski = np.nan # Default to NaN
        
        try:
            # Perform PCA to get the eigenvalues
            # The number of components must be at least N to get all eigenvalues
            n_components = Y_scaled.shape[1]
            pca = PCA(n_components=n_components)
            pca.fit(Y_scaled)
            eigenvalues = pca.explained_variance_
            
            # Apply the Onatski criterion
            num_factors_onatski = onatski_criterion(eigenvalues, n_max=8)

        except ValueError as e:
            print(f"Skipping Onatski for {region.upper()} split {sp}: {e}")

        # Store results
        result = {
            "region": region,
            "split": sp,
            "T": Y_train.shape[0],
            "N": Y_train.shape[1],
            "onatski_factors": num_factors_onatski
        }
        onatski_results_rows.append(result)

# Convert results to a DataFrame for analysis and lookup
onatski_analysis_df = pd.DataFrame(onatski_results_rows).set_index(['region', 'split'])

print("\n--- Optimal Static Factors (k) from Onatski (2010) ---")
print(onatski_analysis_df)

In [ ]:
print("\n--- Running Sensitivity Analysis for Dynamic Factors (q) based on static factors from Onatski method  ---")

# Define parameters for the analysis
regions = ['ch', 'eu', 'us']
split_percentages = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
p_lags = [1, 2, 3] # VAR lags to test
results_rows_dynamic = []

for region in regions:
    data_matrix = multivar_data_win[region].select_dtypes(include=[np.number]).dropna(axis=1, how='all')
    T_total = data_matrix.shape[0]

    for sp in split_percentages:
        split_size = int(T_total * sp)
        Y_train = data_matrix.iloc[:split_size, :].dropna(axis=0)
        T, N = Y_train.shape
        
        if N < 2:
            continue

        # Look up the number of static factors from the Onatski results
        try:
            NF = int(onatski_analysis_df.loc[(region, sp), 'onatski_factors'])
        except (KeyError, ValueError):
            print(f"Warning: Could not find or use Onatski result for {region.upper()} split {sp}. Skipping.")
            continue
        
        # If NF is NaN or less than 1, skip
        if pd.isna(NF) or NF < 1:
            continue
            
        # Pre-fit the scaler for efficiency
        scaler = StandardScaler(with_mean=True, with_std=False).fit(Y_train.values)
        
        # Loop over VAR lags p
        for p in p_lags:
            if T <= max(NF, p) + 5:  # Ensure T is large enough for the VAR model
                continue
            
            # Determine dynamic factors with the specific NF from Onatski
            q1, q2 = dynamic_factors(Y_train.values, NF=NF, p=p, scaler=scaler)
            q = min(q1, q2)

            results_rows_dynamic.append({
                "region": region,
                "split": sp,
                "k_static_factors": NF,
                "p_lag": p,
                "q_dynamic_factors": q,
            })

# --- Analyze and Display the Results ---
dynamic_analysis_df = pd.DataFrame(results_rows_dynamic)

print("\n--- Full Sensitivity Analysis Results ---")

for r in dynamic_analysis_df["region"].unique():
    print(f"\n--- Region: {r.upper()} ---")
    
    region_df = dynamic_analysis_df[dynamic_analysis_df.region == r]
    
    # Pivot to show how q changes with the choice of p_lag
    final_pivot = region_df.pivot_table(
        index=['split', 'k_static_factors'], 
        columns='p_lag', 
        values='q_dynamic_factors'
    )
    print(final_pivot)

In [ ]:
# --- Save the Final Selection ---
TARGET_P_LAG = 1
print(f"\n--- Generating Final Selection File for p_lag = {TARGET_P_LAG} ---")

factor_selection = dynamic_analysis_df[
    dynamic_analysis_df['p_lag'] == TARGET_P_LAG
].copy()

final_columns = {
    'region': 'region',
    'split': 'split',
    'k_static_factors': 'static_factors',
    'q_dynamic_factors': 'dynamic_factors'
}
factor_selection = factor_selection[final_columns.keys()].rename(columns=final_columns)

out_path_final = ROOT_DIR / "results" / "tables" / "factor_selection.csv"
factor_selection.to_csv(out_path_final, index=False)
print(f"\nSaved final factor selection to: '{out_path_final}'")
print(factor_selection)